In [ ]:
import numpy as np
mean_x1 = 1
mean_x2 = -1
mean_x = np.array([mean_x1, mean_x2])

In [ ]:
import danatools
sigma_x1 = 0.056
sigma_x2 = 0.048
correlation =  -0.85
covariance_x = danatools.covariance_matrix_2d(sigma_x1, sigma_x2, correlation)

## Variance propagation

In [ ]:
def function(x1, x2):
    return x1 * np.exp(x2)

In [ ]:
mean_y_ana = function(mean_x1, mean_x2)
mean_y_ana

In [ ]:
import math
gradient_x1 = math.exp(mean_x2)
gradient_x2 = mean_x1 * math.exp(mean_x2)

In [ ]:
variance_y_ana = gradient_x1**2 * covariance_x[0,0] + gradient_x2**2 * covariance_x[1,1] + 2 * gradient_x1 * gradient_x2 *  covariance_x[0,1]
variance_y_ana

In [ ]:
sigma_y_ana = math.sqrt(variance_y_ana)
sigma_y_ana

## Simulations

In [ ]:
from scipy.stats import multivariate_normal
population = 1000000
rng = np.random.default_rng(seed=6870)
data_x = multivariate_normal.rvs(mean_x, covariance_x, size=population, random_state=rng)
data_x

In [ ]:
data_y = function(data_x[:,0], data_x[:,1])
data_y

In [ ]:
mean_y_sim = data_y.mean()
mean_y_sim

In [ ]:
sigma_y_sim = data_y.std(ddof=1)
sigma_y_sim

## Variance propagation vs. simulations

In [ ]:
delta_mean = (mean_y_ana / mean_y_sim - 1) 
print(f"Mean: {delta_mean*100:.2f}%")

In [ ]:
delta_sigma = (sigma_y_ana / sigma_y_sim - 1) 
print(f"Standard deviation: {delta_sigma*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
fig, ax = plt.subplots()
ax.set_xlabel("$X_1$")
ax.set_ylabel("$X_2$")

height_limits = [mean_x1-3*sigma_x1, mean_x1+3*sigma_x1]
weight_limits = [mean_x2-3*sigma_x2, mean_x2+3*sigma_x2]
counts, xedges, yedges, im = ax.hist2d(data_x[:,0], data_x[:,1], range=[height_limits, weight_limits], bins=100, density=True, cmap=cm.viridis)
clb = plt.colorbar(im)

ax.plot(*danatools.get_ellipse(mean_x, covariance_x, nsigma=1), color='tab:red', ls='--')
ax.plot(*danatools.get_ellipse(mean_x, covariance_x, nsigma=2), color='tab:red', ls='--')

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("$Y$")
ax.set_ylabel(" Probability density")

limits = [mean_y_ana-3*sigma_y_ana, mean_y_ana+3*sigma_y_ana] 
ax.hist(data_y, range=limits, bins=100, density=True, label="Simulation")

from scipy.stats import norm
x = np.linspace(*limits, 100)
y = norm.pdf(x, loc=mean_y_ana, scale=sigma_y_ana)
plt.plot(x, y, label="PDF")

ax.legend()